# Example usage of the Lakeshore Model 372 to control the temperature of the Bluefors fridge
The Model 372 is used to control the temperature of the Bluefors fridges. To use it as such outside of the control software provided by Bluefors one has to establish an addtional connection. Within the Bluefors system, the lakeshore is connected via its usb port (through a USB hub along with the other devices) to the control Laptop (as part of the Bluefors setup). To control the temperature of the fridge via QCoDeS it is most convenient to connect the Lakeshore via its LAN port to the control computer (QCoDeS not Bluefors). For this the Bluefors rack has to be opened (Don't disconnect USB). Remember to set the IP address setting to DHCP (of course only when using a router).

For using the Lakeshore with qcodes, the operation mode has to be switched from USB to LAN. When done with the measurements please switch back to USB, so that the logging of the Temperature provided by the Bluefors software continues.
We are already planning to implement a server that takes care of the logging so that switching will no longer be necessary. For the time being please always remember to switch back!

### Driver Setup
This notebook is using a simulated version of the driver so that it can be run and played with, without an actual instrument. When trying it out with a real Lakeshore, please set `simulation = False`.

**Be careful, this notebook has not been tested with a real device yet!**

In [1]:
simulation = True

In [2]:
if simulation:
    from qcodes.tests.drivers.test_lakeshore import Model_372_Mock as Model_372
    import qcodes.instrument.sims as sims
    visalib = sims.__file__.replace('__init__.py',
                                    'lakeshore_model372.yaml@sim')
    ls = Model_372('lakeshore_372', 'GPIB::3::65535::INSTR',
                    visalib=visalib, device_clear=False)
else:
    from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
    ls = Model_372('lakeshore_372', '<put your visa address here, see e.g. NI Max')


C:\Users\a-dovoge\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Connected to: None lakeshore_372 (serial:None, firmware:None) in 0.47s


### Readout Sensor Channels
The lakeshore has two types of *channels*: *Readout channels* and *heaters*. For reading the temperature we use the readout channels. There are seventeen channels each of which has the following parameters:

In [14]:
ls.ch01.parameters

{'temperature': <qcodes.instrument.parameter.Parameter: temperature at 2885538613456>,
 't_limit': <qcodes.instrument.parameter.Parameter: t_limit at 2885538612896>,
 'sensor_raw': <qcodes.instrument.parameter.Parameter: sensor_raw at 2885538613232>,
 'sensor_status': <qcodes.instrument.parameter.Parameter: sensor_status at 2885538614632>,
 'sensor_name': <qcodes.instrument.parameter.Parameter: sensor_name at 2885538614408>}

to read all temperatures we can do the following:

In [4]:
for ch in ls.channels:
    print(f'Temperature of {ch.short_name}: {ch.temperature()} K')

Temperature of ch01: 4.0 K
Temperature of ch02: 4.0 K
Temperature of ch03: 4.0 K
Temperature of ch04: 4.0 K
Temperature of ch05: 4.0 K
Temperature of ch06: 4.0 K
Temperature of ch07: 4.0 K
Temperature of ch08: 4.0 K
Temperature of ch09: 4.0 K
Temperature of ch10: 4.0 K
Temperature of ch11: 4.0 K
Temperature of ch12: 4.0 K
Temperature of ch13: 4.0 K
Temperature of ch14: 4.0 K
Temperature of ch15: 4.0 K
Temperature of ch16: 4.0 K


### Heating and Feedback
To set a certain temperature one needs to start a feedback loop that reads the temperature of a channel and feeds back to the sample through a heater. The lakeshore has three heaters: `sample_heater`, `warmup_heater`, and `analog_heater`.

Here the `sample_heater` will be used. It has the following parameters:

In [5]:
h = ls.sample_heater
h.parameters

{'mode': <qcodes.instrument_drivers.Lakeshore.lakeshore_base.GroupParameter: mode at 2885539082592>,
 'input_channel': <qcodes.instrument_drivers.Lakeshore.lakeshore_base.GroupParameter: input_channel at 2885539082760>,
 'powerup_enable': <qcodes.instrument_drivers.Lakeshore.lakeshore_base.GroupParameter: powerup_enable at 2885539082872>,
 'P': <qcodes.instrument_drivers.Lakeshore.lakeshore_base.GroupParameter: P at 2885539083152>,
 'I': <qcodes.instrument_drivers.Lakeshore.lakeshore_base.GroupParameter: I at 2885539083320>,
 'D': <qcodes.instrument_drivers.Lakeshore.lakeshore_base.GroupParameter: D at 2885539083488>,
 'output_range': <qcodes.instrument.parameter.Parameter: output_range at 2885539083712>,
 'setpoint': <qcodes.instrument.parameter.Parameter: setpoint at 2885539083936>,
 'range_limits': <qcodes.instrument.parameter.Parameter: range_limits at 2885539084496>,
 'wait_cycle_time': <qcodes.instrument.parameter.Parameter: wait_cycle_time at 2885539084552>,
 'wait_tolerance': <

The allowed modes, polarities, and ranges are defined in:

In [6]:
h.MODES

{'off': 0,
 'monitor_out': 1,
 'open_loop': 2,
 'zone': 3,
 'still': 4,
 'closed_loop': 5,
 'warm_up': 6}

In [7]:
h.RANGES

{'off': 0,
 '31.6μA': 1,
 '100μA': 2,
 '316μA': 3,
 '1mA': 4,
 '3.16mA': 5,
 '10mA': 6,
 '31.6mA': 7,
 '100mA': 8}

In [8]:
h.POLARITIES

{'unipolar': 0, 'bipolar': 1}

For the PID mode we need to set the P, I, and D constants, the setpoint and the heater range, and start the operation by setting mode to 'closed_loop'

In [9]:
h.P(10)
h.I(10)
h.D(0)
h.output_range('31.6μA')
h.input_channel(1)

In [10]:
h.setpoint(0.01)
h.mode('closed_loop')

Now we can observe how the temperature gets steered towards the setpoint: (This is not implemented in the simulated instrument)

In [16]:
import time
for i in range(10):
    time.sleep(0.5)
    print(f'T={ls.ch01.temperature()}')

T=4.0
T=4.0
T=4.0
T=4.0
T=4.0
T=4.0
T=4.0
T=4.0
T=4.0
T=4.0


### Waiting to reach setpoint
As we have seen, the call of the parameter `setpoint` is non-blocking. That means the function returns imediately without waiting for the setpoint to be reached. In many use-cases it is desirable to wait until a certain temperature regime has been reached. This can be achieved with `wait_until_set_point_reached()`. 

In [18]:
# wait half a second, then read the temperature and compare to setpoint
h.wait_cycle_time(0.5)
# wait until temperature within 5% of the setpoint
# the tolerance is defined as: |t_reading-t_setpoint|/t_reading
h.wait_tolerance(0.05)
# wait until temperature has been within the tolerance regime for `wait_equilibration_time` seconds
h.wait_equilibration_time(1.5)

# do the waiting:
# !! does not work with simulated instrument !!
h.wait_until_set_point_reached()

loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0


KeyboardInterrupt: 

** Simulation **

In case of a simulation we can fake the heating of the sample by calling the `start_heating` method (which only exists for the simulated instrument)

In [19]:
if simulation:
    ls.sample_heater.setpoint(4)
    ls.start_heating()
    ls.sample_heater.wait_until_set_point_reached()

loop iteration with t reading of 6.998815100000002
loop iteration with t reading of 6.476926400000025
loop iteration with t reading of 5.936238600000024
loop iteration with t reading of 5.334502799999996
loop iteration with t reading of 4.831042600000046
loop iteration with t reading of 4.33076410000001
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0
loop iteration with t reading of 4.0


### Automatically selecting a heater range
To automatically select a heater range one can define temperature limits for the individual heater ranges:

In [23]:
# all limits in K, 7 limits starting with limit for 31.6μA
h.range_limits([0.021, 0.1, 0.2, 1.1, 2, 4, 8])

this means: from 0 K to 0.021 K use 31.6μA, from 0.021 K to 0.1 K use 100μA and so on

we can now set the range by giving a temperature:

In [25]:
h.set_range_from_temperature(0.15)
h.output_range()

'316μA'

### Sweeping/blocking paramameter
For compatibility with the legacy loop construct the lakeshore driver exposes a blocking temperature parameter: `blocking_T`.
The setter for this parameter simply does:

```
def _set_blocking_T(self, T):
     self.set_range_from_temperature(T)
     self.setpoint(T)
     self.wait_until_set_point_reached()
```

This parameter can be used in a doNd loop.

** the range only gets set at the beginning of the sweep, i.e. according to the setpoint not according to the temperature reading **

